In [1]:
!pip3 install folium wget pandas

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=a5cb8172ea6950ebf2e791cb5b1a8b6ddbdae14f8572c2cd9500c297ccc1e3b9
  Stored in directory: c:\users\21418\appdata\local\pip\cache\wheels\01\46\3b\e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [3]:
import folium
import wget
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

In [5]:
# Download and read the dataset
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df = pd.read_csv(spacex_csv_file)

# Select relevant columns and get unique launch sites
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

In [9]:
# Create a map centered around NASA
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add markers for each launch site
for index, site in launch_sites_df.iterrows():
    # Create a circle
    circle = folium.Circle(
        [site['Lat'], site['Long']],
        radius=1000,
        color='#000000',
        fill=True
    ).add_child(folium.Popup(site['Launch Site']))
    
    # Create a marker with label
    marker = folium.Marker(
        [site['Lat'], site['Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{site["Launch Site"]}</b></div>'
        )
    )
    
    # Add to map
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

In [11]:
# Function to assign color based on launch outcome
def assign_marker_color(launch_outcome):
    return 'green' if launch_outcome == 1 else 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [13]:
# Create marker cluster
marker_cluster = MarkerCluster()

# Add to map
site_map.add_child(marker_cluster)

# Add launch outcome markers
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        [record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map

In [15]:
# Add Mouse Position to get coordinates
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [17]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [19]:
# Coordinates for CCAFS SLC-40
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Example coastline point (found using MousePosition)
coastline_lat = 28.56334
coastline_lon = -80.56799

# Calculate distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create distance marker
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>'
    )
)

# Draw line between points
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines = folium.PolyLine(locations=coordinates, weight=1)

# Add to map
site_map.add_child(distance_marker)
site_map.add_child(lines)
site_map

In [21]:
# Example for highway near CCAFS SLC-40
highway_lat = 28.56335
highway_lon = -80.57085
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

highway_marker = folium.Marker(
    [highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance_highway:.2f} KM</b></div>'
    )
)

coordinates = [[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]]
lines = folium.PolyLine(locations=coordinates, weight=1, color='blue')

site_map.add_child(highway_marker)
site_map.add_child(lines)
site_map